In [1]:
import datasets
from datasets import load_dataset
##
import evaluate
##
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments

##
import numpy as np


/home/akash/anaconda3/envs/vision/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-04-16 14:41:54.391591: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-16 14:41:54.499386: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-16 14:41:54.499483: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-16 14:41:54.510949: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugi

In [2]:
model_identifier='google-bert/bert-base-cased'

tokenizer=AutoTokenizer.from_pretrained(pretrained_model_name_or_path=model_identifier
                                      )

In [3]:
### loading 

dataset_name='yelp_review_full'

dataset=load_dataset(dataset_name)

In [4]:
num_label=np.unique(dataset['train']['label']).shape[0]

In [6]:
def tokenize_example(example):
    return (tokenizer(example['text'],padding=True,truncation=True)) ## it will radd features like input_ids,token_type_ids, attention_mask

In [7]:
dataset=dataset.map(tokenize_example,batched=True)

Map: 100%|██████████| 50000/50000 [00:15<00:00, 3218.12 examples/s]


In [8]:
dataset['train']

Dataset({
    features: ['label', 'text', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 650000
})

In [9]:
dataset['train'][0]['input_ids']

[101,
 173,
 1197,
 119,
 2284,
 2953,
 3272,
 1917,
 178,
 1440,
 1111,
 1107,
 170,
 1704,
 22351,
 119,
 1119,
 112,
 188,
 3505,
 1105,
 3123,
 1106,
 2037,
 1106,
 1443,
 1217,
 10063,
 4404,
 132,
 1119,
 112,
 188,
 1579,
 1113,
 1159,
 1107,
 3195,
 1117,
 4420,
 132,
 1119,
 112,
 188,
 6559,
 1114,
 170,
 1499,
 118,
 23555,
 2704,
 113,
 183,
 9379,
 114,
 1134,
 1139,
 2153,
 1138,
 3716,
 1106,
 1143,
 1110,
 1304,
 1696,
 1107,
 1692,
 1380,
 5940,
 1105,
 1128,
 1444,
 6059,
 132,
 1105,
 1128,
 1169,
 1243,
 5991,
 16179,
 1106,
 1267,
 18137,
 1443,
 1515,
 1106,
 1267,
 1140,
 1148,
 119,
 1541,
 117,
 1184,
 1167,
 1202,
 1128,
 1444,
 136,
 178,
 112,
 182,
 2807,
 1303,
 1774,
 1106,
 1341,
 1104,
 1251,
 11344,
 178,
 1138,
 1164,
 1140,
 117,
 1133,
 178,
 112,
 182,
 1541,
 4619,
 170,
 9153,
 119,
 102,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


In [10]:
## create a small portion of training and evaluation dataset
seed=101
small_train_dataset=dataset['train'].shuffle(seed=seed).select(range(1000))
small_eval_dataset=dataset['test'].shuffle(seed=seed).select(range(1000))

In [11]:
small_train_dataset

Dataset({
    features: ['label', 'text', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 1000
})

## learning about evaluate module

Using evaluate library we can load various evaluation metrics such as accuracy. For instance, 
'''python
import 
metrics=evaluate
'''

In [12]:
##loading the metrics
## using the evaluate we can load different type of metrics such as accuracy

metric=evaluate.load('accuracy')




In [13]:

## when we want to get the accuracy we can use metric.compute(predictions=predicted_label,references=True_label)

def compute_metrics(eval_pred): ## these args are passed automaticall when calling Trainer() object later

    logits,labels=eval_pred
    pred_label=np.argmax(logits,axis=1)
    return metric.compute(predictions=pred_label,references=labels)

In [14]:
## next we will call the AutoModel and train the last layer

model=AutoModelForSequenceClassification.from_pretrained(pretrained_model_name_or_path=model_identifier,
                                                         num_labels=num_label)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [15]:
### now we need to call tramsformer.TrainingArguments(); required_args is output_dir; where we wanna overwrite the config
## there are also other config, for now we will use default one. If not provided, it will select default args

training_args=TrainingArguments(output_dir='test_trainer')

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [17]:
### now we will define Trainer

trainer=Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
    
)

/home/akash/anaconda3/envs/vision/lib/python3.9/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [18]:
## training
trainer.train()

Step,Training Loss


TrainOutput(global_step=375, training_loss=0.95759130859375, metrics={'train_runtime': 163.5378, 'train_samples_per_second': 18.344, 'train_steps_per_second': 2.293, 'total_flos': 789354427392000.0, 'train_loss': 0.95759130859375, 'epoch': 3.0})

In [ ]:
###